In [10]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import matplotlib.pyplot as plt
import pandas as pd
from src.model import BasicModel

plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

class LEARN_INFO:
    def __init__(self, name, columns, is_contain_cluster_label=False, matching_type="general", jump=3):
        self.name = name
        self.columns = columns
        self.is_label = is_contain_cluster_label
        self.matching = matching_type
        self.jump = jump
    

# INPUT_REPORT_SIMULATION = [
#     LEARN_INFO(name="univariate", columns=['energy (kw 15min)'], is_contain_cluster_label=False),
#     LEARN_INFO(name="with cluster label", columns=['energy (kw 15min)'], is_contain_cluster_label=True),
# ]

DEFAULT_COLUMNS = [
    'energy (kw 15min)',
    ['week sin',
    'week cos'],
    ['year sin',
    'year cos'],
    'season idx',
    'weather idx',
    'avg ta',
    'avg rhm'
]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos'
    ],
#     "multivariate (year)": [
#         'energy (kw 15min)',
#         'year sin',
#         'year cos',
#     ],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
#     "multivariate (avg ta)": [
#         'energy (kw 15min)',
#         'avg ta'
#     ],
#     "multivariate (avg rhm)": [
#         'energy (kw 15min)',
#         'avg rhm'
#     ],
#     "multivariate (season)": [
#         'energy (kw 15min)',
#         'season idx'
#     ]
}

# Input Report Simulation
IRS = []

for key in _.keys():
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )
    
    
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

# Training Model Array 
TMA = []

is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch,
                   matching_type=_.matching,
                   is_multi_step=True,
                   model_type="Multi Step / SingleShot")
    
    bm.set_window(OUT_STEPS=3)
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_multi_predict(is_reshape=True)
    bm.set_multi_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (weather)) set predict (validation) info start ###### 

0 / 91
50 / 91
91 / 91 complete.

###### [Notice] set predict info success ###### 



In [22]:
test_evaluate_df

,cos,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate / with cluster,0.895798,60.758185,0.289536,1.682935,0.891164,0.159209,1456.910039,571.717939
multivariate (week) / with cluster,0.903986,65.120351,0.268284,1.563445,0.899757,0.141511,1456.910039,508.165102
multivariate (weather) / with cluster,0.898025,61.544063,0.284062,1.659287,0.893040,0.156020,1456.910039,560.268411
univariate,0.884017,60.964155,0.278389,1.688926,0.876994,0.158373,1456.910039,568.717151
multivariate (week),0.892750,63.535232,0.272899,1.621582,0.885165,0.147942,1456.910039,531.258867
multivariate (weather),0.883680,60.920699,0.277681,1.688440,0.876616,0.158549,1456.910039,569.350255


In [23]:
val_evaluate_df

,cos,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate / with cluster,0.933157,73.957148,0.250114,1.465672,0.915018,0.113451,832.494756,216.805374
multivariate (week) / with cluster,0.935891,75.283441,0.241750,1.418462,0.917170,0.107673,832.494756,205.764060
multivariate (weather) / with cluster,0.935426,74.393949,0.247303,1.455741,0.916994,0.111548,832.494756,213.169031
univariate,0.907692,66.235098,0.264690,1.658611,0.887744,0.147091,832.494756,281.091039
multivariate (week),0.908152,66.390871,0.273506,1.633825,0.887997,0.146412,832.494756,279.794237
multivariate (weather),0.908579,66.543006,0.261042,1.648695,0.887669,0.145750,832.494756,278.527722


In [18]:
_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
        'energy (kw 15min)',
        'week sin',
        'week cos'
    ],
#     "multivariate (year)": [
#         'energy (kw 15min)',
#         'year sin',
#         'year cos',
#     ],
    "multivariate (weather)": [
        'energy (kw 15min)',
        'weather idx'
    ],
#     "multivariate (avg ta)": [
#         'energy (kw 15min)',
#         'avg ta'
#     ],
#     "multivariate (avg rhm)": [
#         'energy (kw 15min)',
#         'avg rhm'
#     ],
#     "multivariate (season)": [
#         'energy (kw 15min)',
#         'season idx'
#     ]
}

# Input Report Simulation
IRS = []

for key in _.keys():
    IRS.append(
        LEARN_INFO(name="{} / with cluster".format(key), 
                   columns=_[key], 
                   is_contain_cluster_label=True),
    )
    
    
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

# Training Model Array 
TMA = []

is_switch = True

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch,
                   matching_type=_.matching,
                   is_multi_step=True,
                   model_type="Multi Step / Autoregressive")
    
    bm.set_window(OUT_STEPS=3)
    bm.set_model()
    print(bm)
    bm.training(epochs=50)
    
    bm.set_multi_predict(is_reshape=True)
    bm.set_multi_predict(is_reshape=True, is_val_datas=True)
    
    TMA.append(bm)
    
test_evaluate_df = pd.DataFrame();
val_evaluate_df = pd.DataFrame();

for _ in TMA:
    test_evaluate_df = test_evaluate_df.append(_.statistic(), ignore_index=True)
    val_evaluate_df = val_evaluate_df.append(_.statistic(is_val_datas=True), ignore_index=True)
    
test_evaluate_df.index = [_.name for _ in TMA]
val_evaluate_df.index = [_.name for _ in TMA]

###### [Notice] (multivariate (weather)) set predict (validation) info start ###### 

0 / 171
50 / 171
100 / 171
150 / 171
171 / 171 complete.

###### [Notice] set predict info success ###### 



In [19]:
test_evaluate_df

,cos,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate / with cluster,0.899183,63.163517,0.318809,1.768196,0.878693,0.160472,832.494756,306.661788
multivariate (week) / with cluster,0.889890,59.925819,0.335927,1.831443,0.866012,0.174576,832.494756,333.615458
multivariate (weather) / with cluster,0.894468,61.508055,0.328000,1.807006,0.876751,0.167684,832.494756,320.443421
univariate,0.906928,65.964200,0.261876,1.663649,0.884381,0.148271,832.494756,283.346253
multivariate (week),0.911467,67.529879,0.260056,1.601386,0.889188,0.141451,832.494756,270.312051
multivariate (weather),0.906689,65.880605,0.261479,1.664525,0.884507,0.148635,832.494756,284.042174


In [20]:
val_evaluate_df

,cos,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate / with cluster,0.876867,54.823534,0.321133,1.829505,0.870987,0.183286,1456.910039,658.180474
multivariate (week) / with cluster,0.876151,55.079528,0.320149,1.809671,0.869455,0.182248,1456.910039,654.450868
multivariate (weather) / with cluster,0.872699,53.793156,0.324950,1.852496,0.867317,0.187466,1456.910039,673.192144
univariate,0.885588,61.496197,0.274808,1.676737,0.878777,0.156214,1456.910039,560.965775
multivariate (week),0.895579,64.551933,0.263922,1.584380,0.889120,0.143817,1456.910039,516.446450
multivariate (weather),0.884615,61.233698,0.274991,1.679640,0.878309,0.157279,1456.910039,564.790152
